In [ ]:
'''
Created: Trent Nguyen just.trent.n@gmail.com

Data analyst on crimes in Orlando city limit.  Data obtained from Orlando Police Department.
'''
import gmaps
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import calendar


In [ ]:
# !!! must have  google maps api key
gmaps.configure(api_key='')


In [ ]:
# https://data.cityoforlando.net/Orlando-Police/OPD-Crimes/4y9m-jbmz

# This dataset comes from the Orlando Police Department records management system. 
# It includes all Part 1 and Part 2 crimes as defined by the FBI’s Uniform Crime Reporting standards. 
# When multiple crimes are committed the highest level of crime is what is shown. 
# The data includes only Open and Closed cases and does not include informational cases without arrests.
# This data excludes crimes where the victim or offender data is or could be legally protected. 
# Those include:

# Domestic Violence
# Elderly Abuse
# Exempt from Public Record
# Witness Protection
# Cases where the offender was charged with the following case codes/state statutes:
# Sexual Battery
# Child Abuse
# Aggravated Stalking/Harassment
# Obscene or harassing telephone calls
# Aggravated Battery
# Domestic Violence Injunctions
# Lewd,Lascivious or Indecent Assault upon or in the presence of a child
# Elder Abuse
# Baker Act
# Obscenity
# Forcible Rape
# Forcible Sodomy
# Forcible Fondling
# Cases where the a juvenile is arrested 

data_orig = pd.read_csv('OPD_Crimes.csv')
data_orig.head()

## Data Cleaning

In [ ]:
# copy original data frame so cleaning can be done
data1 = data_orig.copy()

In [ ]:
# convert str 'Case Date Time' to Timestamp type
# list comprehension but still slow; over 2 minutes, 
# wonder if there's faster ways to convert to Timestamp
#data1['Case Date Time'] = pd.to_datetime(data1['Case Date Time'])
data1['Case Date Time'] = [pd.to_datetime(x) for x in data1['Case Date Time']]
data1.head()

In [ ]:
# remove parenthensis '()' from Location str
data1['Location'].replace('[\(\)]','', regex=True, inplace=True)

data1.head()

In [ ]:
# drop any rows that contains NaN; mostly in the Location column
data1.dropna(inplace=True)
data1.info()

In [ ]:
# convert the str 'Location' coordinate to float64
location =[ (np.float64(lat), np.float64(long)) for (lat, long) in data1['Location'].str.split(',')]
data1['Location'] = location
data1.head()


In [ ]:
#data1['Case Offense Category'].value_counts()

## Heatmap with interactive years slider
#### Use location and date data to draw heatmap with slider for years

In [ ]:
# Just need the Date and Location for the heatmap
dataForHeatMap = data1[['Case Date Time', 'Location']]

dataForHeatMap = dataForHeatMap.rename(columns={'Case Date Time': 'Date'})
dataForHeatMap.head()


In [ ]:
from HeatMap import clHeatMap
from ipywidgets.embed import embed_minimal_html

In [ ]:
interactiveHeatMap = clHeatMap(dataForHeatMap, 'Date', 'Location')

In [ ]:
interactiveHeatMap.Render()

# embed_minimal_html('OrlHeatMap.html', views=[aaa])

# STOP HERE FOR NOW =====================================

### Get Data on Year 2017

In [ ]:
# get data for yr 2017 for now
bYear2017 =  data1['Case Date Time'].dt.year == 2017

# checked the Location data for 'NaN' (null); 
# can't rely on 'Status' column as it might not be accurate
bLoc = data1['Location'].isnull() == False

# get data for yr 2017 and where location data is available
data2017 = data1[bYear2017 & bLoc]

In [ ]:
data2017.info()

In [ ]:
data2017.head()

### Functions

In [ ]:
def DatFrameCat(df: pd.DataFrame, col_name: str, values_nm: [str] ) -> pd.DataFrame:
    '''
    Return a data frame based on search of values name on a particular column.
    df: pass in data frame
    col_name: column name
    values_nm: values list use to search on column name
    '''    
    bSelect = df[col_name].isin(values_nm)
    ret = df[bSelect]
    
    return ret



In [ ]:
def DrawHeatMap(layer_location) -> gmaps.figure:
    try:
        heatmap = gmaps.figure()
        heatlayer = gmaps.heatmap_layer(layer_location)
        heatmap.add_layer(heatlayer)
        return heatmap
    except:
        pass

### Categorize crime types

In [ ]:
# Larceny = 'Theft','Burglary','Robbery'
Larceny = DatFrameCat(data2017, 'Case Offense Category',['Theft', 'Burglary', 'Robbery'] )
Larceny['Case Offense Category'].value_counts()

In [ ]:
# Assault
Assault = DatFrameCat(data2017, 'Case Offense Category',['Assault'])
Assault['Case Offense Category'].value_counts()


In [ ]:
# Narcotics
Narcotics = DatFrameCat(data2017, 'Case Offense Category',['Narcotics'])
Narcotics['Case Offense Category'].value_counts()


In [ ]:
# Fraud = 'Fraud', 'Embezzlement'
Frauds = DatFrameCat(data2017, 'Case Offense Category',['Fraud', 'Embezzlement', 'Bribery'])
Frauds['Case Offense Category'].value_counts()


In [ ]:
# Vehicle Theft
VThefts  = DatFrameCat(data2017, 'Case Offense Category',['Vehicle Theft'])
VThefts['Case Offense Category'].value_counts()


In [ ]:
# Homicide
Homicide  = DatFrameCat(data2017, 'Case Offense Category',['Homicide'])
Homicide['Case Offense Category'].value_counts()



In [ ]:
# Arson
Arson  = DatFrameCat(data2017, 'Case Offense Category',['Arson'])
Arson['Case Offense Category'].value_counts()


In [ ]:
# Kidnapping
Kidnapping  = DatFrameCat(data2017, 'Case Offense Category',['Kidnapping'])
Kidnapping['Case Offense Category'].value_counts()



In [ ]:
# map1 = gmaps.figure()
# layer1 = gmaps.symbol_layer(Larceny['GPS-Location'], fill_color='#7FFF00', stroke_color='#7FFF00')
# layer2 = gmaps.symbol_layer(Assault['GPS-Location'], fill_color='#00FFFF', stroke_color='#00FFFF')
# layer3 = gmaps.symbol_layer(Narcotics['GPS-Location'], fill_color='#00008B', stroke_color='#00008B')
# layer4 = gmaps.symbol_layer(Fraud['GPS-Location'], fill_color='#006400', stroke_color='#006400')

# map1.add_layer(layer1)
# map1.add_layer(layer2)
# map1.add_layer(layer3)
# map1.add_layer(layer4)

# map1

## Heat Maps

### Orlando Crime:  All Crimes 2017

In [ ]:
All2017heatmap = DrawHeatMap(data2017['Location'])
All2017heatmap

### Orlando Crimes:  Larceny (Theft, Burglary, Robbery) 2017

In [ ]:
LarcenyMap = DrawHeatMap(Larceny['Location'])
LarcenyMap

### Orlando Crime: Assaults in 2017

In [ ]:
AssaultMap = DrawHeatMap(Assault['Location'])
AssaultMap

### Orlando Crime: Frauds in 2017

In [ ]:
FMap = DrawHeatMap(Frauds['Location'])
FMap

### Orlando Crime: Vehicle Theft in 2017

In [ ]:
VTheftmap = DrawHeatMap(VThefts['Location'])
VTheftmap

### Orlando Crime: Homicide in 2017

In [ ]:
Hmap = DrawHeatMap(Homicide['Location'])
Hmap.

### Graphs

In [ ]:
# Hours that were reported for the crimes, not actual hours when crimes were committed
plt.hist(data2017['Case Date Time'].dt.hour, edgecolor='black', bins=(np.arange(25)-.5), rwidth=.8)

plt.title('A Day in Crimes', fontsize=18)
plt.xlabel('Hours')
plt.ylabel('Crimes')

In [ ]:
# Hours that were reported for the crimes, not actual hours when crimes were committed
plt.hist(data2017['Case Date Time'].dt.weekday, color='green', edgecolor='black', bins=np.arange(8)-.5, rwidth=.9)

plt.title('A Week in Crimes', fontsize=18)
plt.xlabel('Days')
plt.ylabel('Crimes')
plt.xticks(range(7), labels=calendar.day_abbr[0:7], fontsize=10)




In [ ]:
# Hours that were reported for the crimes, not actual hours when crimes were committed
plt.hist(data2017['Case Date Time'].dt.month, color='orange', edgecolor='black', bins=(np.arange(14)-.5))

plt.title('A Year in Crimes', fontsize=18)
plt.xlabel('Months')
plt.ylabel('Crimes')
plt.xticks(range(1,13), labels=calendar.month_abbr[1:13])




### Test for Widgets: sliders and links

In [ ]:
import ipywidgets as widgets

a = widgets.IntSlider(description="Delayed", value=2011, min=2011, max=2016 , step=1, continuous_update=True)
b = widgets.IntText(description="Delayed", value=a.value, continuous_update=False)

c = widgets.IntSlider(description="Continuous", continuous_update=True)
d = widgets.IntText(description="Continuous", continuous_update=True)


def OnYearChange(change):    
    b.value =change['new']
    
a.observe(OnYearChange, names='value')

#widgets.link((a, 'value'), (b, 'value'))
#widgets.link((a, 'value'), (c, 'value'))
widgets.link((c, 'value'), (d, 'value'))
widgets.VBox([a,b,c,d])

    